In [1]:
# Chosen dataset
# https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Electronics_v1_00.tsv.gz


In [2]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:6 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:7 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:10 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Reading package lists... Done


In [3]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-05-14 00:00:28--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.1’

postgresql-42.2.9.j 100%[===================>] 892.61K  4.46MB/s    in 0.2s    

2022-05-14 00:00:29 (4.46 MB/s) - ‘postgresql-42.2.9.jar.1’ saved [914037/914037]



In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

# Extract Data

In [7]:
from pyspark import SparkFiles
# Load in Amazon reviews electronics from S3 into a DataFrame
# url = "https://<bucket name>.s3.amazonaws.com/employee.csv"
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Electronics_v1_00.tsv.gz"
spark.sparkContext.addFile(url)

df = spark.read.option('header', 'true').csv(SparkFiles.get("amazon_reviews_us_Electronics_v1_00.tsv.gz"), inferSchema=True, sep='\t', timestampFormat="mm/dd/yy")
df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   41409413|R2MTG1GCZLR2DK|B00428R89M|     112201306|yoomall 5M Antenn...|     Electronics|          5|            0|          0|   N|                Y|          Five Stars|       As described.| 2015-08-31|
|         US|   49668221|R2HBOEM8LE9928|B000068O48|     734576678|Hosa GPM-103 3.5m...|     Electronics|          5|    

In [18]:
# Create view to make spark sql available to use
df.createOrReplaceTempView("electronics")

In [13]:
# Change the type of review_date to date type
from pyspark.sql.functions import to_date

df = df.withColumn("review_date", to_date(df["review_date"], "yyyy-MM-dd"))
df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: date (nullable = true)



In [14]:
# Count the number of records in the dataset
df.count()

3093869

# Transform Data

## customers_table
customer_id (PK) (integer)<br>
customer_count (integer)<br>

In [35]:
# Create customers_table
customers_table = spark.sql("select customer_id, count(customer_id) as customer_count from electronics group by customer_id")
customers_table.show()
customers_table.printSchema()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   42560427|             6|
|   28871066|             1|
|    6711334|             2|
|   46393060|             1|
|   48101124|             1|
|   38209321|             1|
|   47108763|             4|
|   43252205|             1|
|   41283385|             1|
|   46909180|             5|
|    5453476|             2|
|   12406466|             3|
|   48146680|             1|
|    5953456|             1|
|   16457323|             5|
|   24540309|             1|
|   43366878|             2|
|   28796717|             1|
|   28258386|             1|
|   26079415|             2|
+-----------+--------------+
only showing top 20 rows

root
 |-- customer_id: integer (nullable = true)
 |-- customer_count: long (nullable = false)



In [38]:
customers_table = customers_table.withColumn("customer_count", customers_table["customer_count"].cast("integer"))
customers_table.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- customer_count: integer (nullable = false)



In [51]:
spark.sql("select count(*) from electronics").show()

+--------+
|count(1)|
+--------+
| 3093869|
+--------+



## products_table
product_id (PK) (string)<br>
product_title (string)<br>

In [34]:
# Create products_table
products_table = spark.sql("select product_id, product_title from electronics")
products_table.show()
products_table.printSchema()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B00428R89M|yoomall 5M Antenn...|
|B000068O48|Hosa GPM-103 3.5m...|
|B000GGKOG8|Channel Master Ti...|
|B000NU4OTA|LIMTECH Wall char...|
|B00JOQIO6S|Skullcandy Air Ra...|
|B008NCD2LG|Pioneer SP-BS22-L...|
|B00007FGUF|C2G/Cables to Go ...|
|B00M9V2RMM|COOLEAD-HDMI Swit...|
|B00J3O9DYI|Philips Wireless ...|
|B00NS1A0E4|PlayStation 3 3D ...|
|B007B5V092|JVC HAFR201A Xtre...|
|B00IODHGVG|Sylvania Alarm Cl...|
|B0035PBHX6|Coby 8 GB 1.8-Inc...|
|B00K1JJWFO|Diamond (Original...|
|B00S3LJ5EA|Kingvom 8gb 50 Ho...|
|B00B5QNGN6|JBL Ultra-Portabl...|
|B00PLE8QF6|YIPBOWPT Surface ...|
|B00GHUSIU6|StarTech.com Mini...|
|B0045EJY90|TEAC CD-P650-B Co...|
|B003BT6BM8|Philips SHS8100/2...|
+----------+--------------------+
only showing top 20 rows

root
 |-- product_id: string (nullable = true)
 |-- product_title: string (nullable = true)



## review_id_table
review_id (PK) (string)<br>
customer_id (integer)<br>
product_id (string)<br>
product_parent (integer)<br>
review_date (date)<br>

In [33]:
# Create review_id_table
review_id_table = spark.sql("select review_id, customer_id, product_id, product_parent, review_date from electronics")
review_id_table.show()
review_id_table.printSchema()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R2MTG1GCZLR2DK|   41409413|B00428R89M|     112201306| 2015-08-31|
|R2HBOEM8LE9928|   49668221|B000068O48|     734576678| 2015-08-31|
|R1P4RW1R9FDPEE|   12338275|B000GGKOG8|     614448099| 2015-08-31|
|R1EBPM82ENI67M|   38487968|B000NU4OTA|      72265257| 2015-08-31|
|R372S58V6D11AT|   23732619|B00JOQIO6S|     308169188| 2015-08-31|
|R1A4514XOYI1PD|   21257820|B008NCD2LG|     976385982| 2015-08-31|
|R20D9EHB7N20V6|    3084991|B00007FGUF|     670878953| 2015-08-31|
|R1WUTD8MVSROJU|    8153674|B00M9V2RMM|     508452933| 2015-08-31|
|R1QCYLT25812DM|   52246189|B00J3O9DYI|     766372886| 2015-08-31|
| R904DQPBCEM7A|   41463864|B00NS1A0E4|     458130381| 2015-08-31|
|R1DGA6UQIVLKZ7|    2781942|B007B5V092|     152891509| 2015-08-31|
| RLQT3V8SMNIBH|     707292|B00IODHGVG|     717335277| 2015-08

## vine_table
review_id (PK) (string)<br>
star_rating (integer)<br>
helpful_votes (integer)<br>
total_votes (integer)<br>
vine (string)<br>

In [32]:
# Create vine_table
vine_table = spark.sql("select review_id, star_rating, helpful_votes, total_votes, vine from electronics")
vine_table.show()
vine_table.printSchema()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R2MTG1GCZLR2DK|          5|            0|          0|   N|
|R2HBOEM8LE9928|          5|            0|          0|   N|
|R1P4RW1R9FDPEE|          5|            1|          1|   N|
|R1EBPM82ENI67M|          1|            0|          0|   N|
|R372S58V6D11AT|          5|            1|          1|   N|
|R1A4514XOYI1PD|          5|            1|          1|   N|
|R20D9EHB7N20V6|          5|            0|          0|   N|
|R1WUTD8MVSROJU|          5|            0|          0|   N|
|R1QCYLT25812DM|          4|            0|          0|   N|
| R904DQPBCEM7A|          4|            0|          0|   N|
|R1DGA6UQIVLKZ7|          5|            0|          0|   N|
| RLQT3V8SMNIBH|          5|            0|          0|   N|
|R3T9GZS2TMXZGM|          1|            0|          0|   N|
|R24HVAEYP5PLDN|          5|            

# Load Data

In [41]:
# Configure settings for RDS
mode = "append"
# jdbc_url="jdbc:postgresql://<connection string>:5432/<database-name>"
jdbc_url="jdbc:postgresql://database-bigdata.czywd26sbfeu.us-west-2.rds.amazonaws.com:5432/datasetlvl1"

config = {"user":"postgres", 
          "password": "postgres", 
          "driver":"org.postgresql.Driver"}

In [42]:
# Write DataFrame to customers_table table in RDS
customers_table.write.jdbc(url = jdbc_url, table = 'customers', mode = mode, properties = config)

In [55]:
# Write DataFrame to products_table table in RDS
products_table.write.jdbc(url = jdbc_url, table = 'products', mode = "overwrite", properties = config)

In [56]:
# Write DataFrame to review_id_table table in RDS
review_id_table.write.jdbc(url = jdbc_url, table = 'review_id_table', mode = mode, properties = config)

In [57]:
# Write DataFrame to vine_table table in RDS
vine_table.write.jdbc(url = jdbc_url, table = 'vine_table', mode = mode, properties = config)